Authors: A.Radhakrishnan, GFDL
Ack: Anderson Banihirwe for intake-esm updates, GFDL colleagues for data

esgf-world.json is the ESM collections spec file for the netCDF data in the S3 bucket esgf-world. 
The catalog is updated on an on-demand basis for now. 
You can refer to https://github.com/aradhakrishnanGFDL/gfdl-aws-analysis/tree/community/esm-collection-spec-examples for the most recent catalogs
More examples can be found in https://github.com/aradhakrishnanGFDL/gfdl-aws-analysis/tree/community/examples 


In [6]:
col_url = "https://cmip6-nc.s3.us-east-2.amazonaws.com/esgf-world.json" 

In [ ]:
from glob import glob
import xarray as xr
import cftime
import numpy as np
import matplotlib.pyplot as plt
import intake, intake_esm
import pandas as pd
pd.set_option("display.max_colwidth", None)
#!pip install cmip6_preprocessing

%pip install git+https://github.com/jbusecke/cmip6_preprocessing.git


from cmip6_preprocessing.preprocessing import combined_preprocessing
from cmip6_preprocessing.preprocessing import (correct_units,rename_cmip6)

In [8]:
col = intake.open_esm_datastore(col_url)
esmcol_data = col.esmcol_data

In [9]:
def latest_version(cat):
    """
    input
    cat: esmdatastore
    output
    esmdatastore with latest DRS versions
    """
    latest_cat = cat.df.sort_values(by=['version','path']).drop_duplicates(['temporal subset','model','mip_table',
                                               'institute','variable','ensemble_member',
                                               'grid_label','experiment_id'],keep='last')
    return latest_cat

In [10]:
query_Omon_zos = dict(experiment_id=['abrupt-4xCO2'],#,'historical'],
                 mip_table=['Omon'],
                 ensemble_member=["r1i1p1f1"],
                 model=['IPSL-CM6A-LR'],#,'GFDL-ESM4'],
                 grid_label=['gn'],
                 variable=["zos"])

cat_Omon_zos = col.search(**query_Omon_zos)
cat_Omon_zos_lat = latest_version(cat_Omon_zos)


cat_Omon_zos_latest = intake.open_esm_datastore(cat_Omon_zos_lat,esmcol_data=esmcol_data)

In [11]:
cat_Omon_zos_latest.df

,project,institute,model,experiment_id,frequency,modeling_realm,mip_table,ensemble_member,grid_label,variable,temporal subset,version,path
3,CMIP6,IPSL,IPSL-CM6A-LR,abrupt-4xCO2,mon,ocean,Omon,r1i1p1f1,gn,zos,185001-214912,v20190118,s3://esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_185001-214912.nc
4,CMIP6,IPSL,IPSL-CM6A-LR,abrupt-4xCO2,mon,ocean,Omon,r1i1p1f1,gn,zos,215001-244912,v20190118,s3://esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_215001-244912.nc
5,CMIP6,IPSL,IPSL-CM6A-LR,abrupt-4xCO2,mon,ocean,Omon,r1i1p1f1,gn,zos,245001-274912,v20190118,s3://esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_245001-274912.nc


In [85]:
# Pass this function for preprocessing zos data if needed
def pp_zos(ds):
    ds = rename_cmip6(ds)
  #  ds = fix_time(ds)
    ds = correct_units(ds)
    return ds

TEST 1 - fails on ds_zos_ipsl_abrupt.sel(time=slice("2150","2151"))

In [12]:
dset_dict_zos = cat_Omon_zos_latest.to_dataset_dict(cdf_kwargs={'decode_times': False,'chunks': {'time': 1}}, storage_options={'anon':True},
                                                   )



--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


In [13]:
ds_zos_ipsl_abrupt = dset_dict_zos["CMIP6.IPSL.IPSL-CM6A-LR.abrupt-4xCO2.Omon"]


In [14]:
ds_zos_ipsl_abrupt

<xarray.Dataset>
Dimensions:          (axis_nbounds: 2, ensemble_member: 1, nvertex: 4, time: 10800, x: 362, y: 332)
Coordinates:
    nav_lat          (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
    nav_lon          (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
  * time             (time) float64 15.5 45.0 74.5 ... 1.095e+05 1.096e+05
  * ensemble_member  (ensemble_member) <U8 'r1i1p1f1'
Dimensions without coordinates: axis_nbounds, nvertex, x, y
Data variables:
    bounds_nav_lon   (y, x, nvertex) float32 dask.array<chunksize=(332, 362, 4), meta=np.ndarray>
    bounds_nav_lat   (y, x, nvertex) float32 dask.array<chunksize=(332, 362, 4), meta=np.ndarray>
    area             (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
    time_bounds      (time, axis_nbounds) float64 dask.array<chunksize=(1, 2), meta=np.ndarray>
    zos              (ensemble_member, time, y, x) float32 dask.array<chunksize=(1, 1, 332, 362), meta=np.ndarray>
Attributes:
    external_variables:      areacello
    description:             DECK: abrupt-4xCO2
    parent_activity_id:      CMIP
    activity_id:             CMIP
    name:                    /ccc/work/cont003/gencmip6/p86caub/IGCM_OUT/IPSL...
    initialization_index:    [1]
    variable_id:             zos
    grid_label:              gn
    variant_info:            . Information provided by this attribute may in ...
    frequency:               mon
    CMIP6_CV_version:        cv=6.2.3.5-2-g63b123e
    mip_era:                 CMIP6
    sub_experiment:          none
    EXPID:                   abrupt-4xCO2
    source_id:               IPSL-CM6A-LR
    branch_method:           standard
    institution:             Institut Pierre Simon Laplace, Paris 75252, France
    sub_experiment_id:       none
    contact:                 ipsl-cmip6@listes.ipsl.fr
    branch_time_in_parent:   [7305.]
    physics_index:           [1]
    experiment:              abrupt quadrupling of CO2
    realization_index:       [1]
    realm:                   ocean
    branch_time_in_child:    [0.]
    dr2xml_md5sum:           c2dce418e78ca835be1e2ff817c2c403
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.IPSL.IPSL-C...
    parent_experiment_id:    piControl
    tracking_id:             hdl:21.14100/f42c5f9e-d130-4df1-b658-585116192b2...
    product:                 model-output
    parent_time_units:       days since 1850-01-01 00:00:00
    parent_variant_label:    r1i1p1f1
    model_version:           6.1.8
    experiment_id:           abrupt-4xCO2
    table_id:                Omon
    source:                  IPSL-CM6A-LR (2017):  atmos: LMDZ (NPv6, N96; 14...
    parent_source_id:        IPSL-CM6A-LR
    dr2xml_version:          1.16
    title:                   IPSL-CM6A-LR model output prepared for CMIP6 / C...
    parent_mip_era:          CMIP6
    history:                 Thu Jul 26 18:47:36 2018: ncatted -O -a tracking...
    grid:                    native ocean tri-polar grid with 105 k ocean cells
    license:                 CMIP6 model data produced by IPSL is licensed un...
    creation_date:           2018-12-08T02:01:29Z
    variant_label:           r1i1p1f1
    institution_id:          IPSL
    Conventions:             CF-1.7 CMIP-6.2
    nominal_resolution:      100 km
    forcing_index:           [1]
    intake_esm_varname:      ['zos']
    source_type:             AOGCM BGC
    intake_esm_dataset_key:  CMIP6.IPSL.IPSL-CM6A-LR.abrupt-4xCO2.Omon

In [23]:
ds_zos_ipsl_abrupt.sel(time=slice("2150","2151"))


KeyError: '2150'

In [19]:
ds_zos_ipsl_abrupt.time

<xarray.DataArray 'time' (time: 10800)>
array([1.550000e+01, 4.500000e+01, 7.450000e+01, ..., 1.094955e+05,
       1.095260e+05, 1.095565e+05])
Coordinates:
  * time     (time) float64 15.5 45.0 74.5 ... 1.095e+05 1.095e+05 1.096e+05
Attributes:
    axis:           T
    standard_name:  time
    long_name:      Time axis
    calendar:       gregorian
    bounds:         time_bounds

TEST 2 - fails on ds_zos_ipsl_abrupt_t.sel(time=slice("2150","2151"))

In [20]:
dset_dict_zos = cat_Omon_zos_latest.to_dataset_dict(cdf_kwargs={'decode_times': True,'chunks': {'time': 1}}, storage_options={'anon':True},
                                                   )



--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.8/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/tim

In [24]:
ds_zos_ipsl_abrupt_t = dset_dict_zos["CMIP6.IPSL.IPSL-CM6A-LR.abrupt-4xCO2.Omon"]


In [25]:
ds_zos_ipsl_abrupt_t.sel(time=slice("2150","2151"))

TypeError: '<' not supported between instances of 'cftime._cftime.DatetimeGregorian' and 'str'

TEST 3 - works directly using xarray/fsspec on one object with open_dataset and open_mfdataset ds_zos_ipsl_abrupt.sel(time=slice("2150","2151"))

In [ ]:
import xarray as xr
import s3fs

fs_s3 = s3fs.S3FileSystem()


In [27]:
xr.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.8.6 | packaged by conda-forge | (default, Oct  7 2020, 19:08:05) 
[GCC 7.5.0]
python-bits: 64
OS: Linux
OS-release: 4.14.219-161.340.amzn2.x86_64
machine: x86_64
processor: x86_64
byteorder: little
LC_ALL: C.UTF-8
LANG: C.UTF-8
LOCALE: en_US.UTF-8
libhdf5: 1.10.6
libnetcdf: 4.7.4

xarray: 0.16.1
pandas: 1.1.4
numpy: 1.19.4
scipy: 1.5.3
netCDF4: 1.5.4
pydap: installed
h5netcdf: 0.8.1
h5py: 2.10.0
Nio: None
zarr: 2.5.0
cftime: 1.2.1
nc_time_axis: 1.2.0
PseudoNetCDF: None
rasterio: 1.1.8
cfgrib: 0.9.8.4
iris: None
bottleneck: 1.3.2
dask: 2.30.0
distributed: 2.30.1
matplotlib: 3.3.2
cartopy: 0.18.0
seaborn: None
numbagg: None
pint: 0.16.1
setuptools: 49.6.0.post20201009
pip: 20.2.4
conda: None
pytest: None
IPython: 7.19.0
sphinx: 3.3.0


In [54]:
s3path = "s3://esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_215001-244912.nc"
ds = fs_s3.open(s3path, mode='rb')
xr_ds =xr.open_dataset(ds) #works 
xr_ds.zos.sel(time= slice("2150","2151")) 

/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.8/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


<xarray.DataArray 'zos' (time: 24, y: 332, x: 362)>
[2884416 values with dtype=float32]
Coordinates:
    nav_lat  (y, x) float32 ...
    nav_lon  (y, x) float32 ...
  * time     (time) object 2150-01-16 12:00:00 ... 2151-12-16 12:00:00
Dimensions without coordinates: y, x
Attributes:
    standard_name:       sea_surface_height_above_geoid
    long_name:           Sea Surface Height Above Geoid
    units:               m
    online_operation:    average
    cell_methods:        area: mean where sea time: mean
    interval_operation:  2700 s
    interval_write:      1 month
    cell_measures:       area: areacello
    description:         This is the dynamic sea level, so should have zero g...
    history:             none

In [64]:
xr_ds.time

<xarray.DataArray 'time' (time: 3600)>
array([cftime.DatetimeGregorian(2150, 1, 16, 12, 0, 0, 0),
       cftime.DatetimeGregorian(2150, 2, 15, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2150, 3, 16, 12, 0, 0, 0), ...,
       cftime.DatetimeGregorian(2449, 10, 16, 12, 0, 0, 0),
       cftime.DatetimeGregorian(2449, 11, 16, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2449, 12, 16, 12, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 2150-01-16 12:00:00 ... 2449-12-16 12:00:00
Attributes:
    axis:           T
    standard_name:  time
    long_name:      Time axis
    time_origin:    2150-01-01 00:00:00
    bounds:         time_bounds

In [67]:
dataset = []
dataset.append(ds)
#215001-244912
#245001-274912
index1 = 185001
index2 = 214912

while index1 <= 245001:
    index1 += 30000
    if index1 > 245001:
        break
    index2 += 30000
    s3path = 's3://esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_' + str(index1) + '-' + str(index2) + '.nc'
    remote_file = fs_s3.open(s3path, mode='rb')
    dataset.append(remote_file)
    
ds = xr.open_mfdataset(dataset, chunks={'time': 1}, combine='by_coords')

/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.8/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/tim

In [68]:
ds.zos.sel(time= slice("2749","2749"))

<xarray.DataArray 'zos' (time: 12, y: 332, x: 362)>
dask.array<getitem, shape=(12, 332, 362), dtype=float32, chunksize=(1, 332, 362), chunktype=numpy.ndarray>
Coordinates:
    nav_lat  (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
    nav_lon  (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
  * time     (time) object 2749-01-16 12:00:00 ... 2749-12-16 12:00:00
Dimensions without coordinates: y, x
Attributes:
    standard_name:       sea_surface_height_above_geoid
    long_name:           Sea Surface Height Above Geoid
    units:               m
    online_operation:    average
    cell_methods:        area: mean where sea time: mean
    interval_operation:  2700 s
    interval_write:      1 month
    cell_measures:       area: areacello
    description:         This is the dynamic sea level, so should have zero g...
    history:             none

In [70]:
ds['time']

<xarray.DataArray 'time' (time: 7200)>
array([cftime.DatetimeGregorian(2150, 1, 16, 12, 0, 0, 0),
       cftime.DatetimeGregorian(2150, 2, 15, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2150, 3, 16, 12, 0, 0, 0), ...,
       cftime.DatetimeGregorian(2749, 10, 16, 12, 0, 0, 0),
       cftime.DatetimeGregorian(2749, 11, 16, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2749, 12, 16, 12, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 2150-01-16 12:00:00 ... 2749-12-16 12:00:00
Attributes:
    axis:           T
    standard_name:  time
    long_name:      Time axis
    bounds:         time_bounds

In [ ]:
%pip install git+https://github.com/intake/intake-esm.git


In [35]:
import intake, intake_esm
col = intake.open_esm_datastore(col_url)
esmcol_data = col.esmcol_data

query_Omon_zos = dict(experiment_id=['abrupt-4xCO2'],#,'historical'],
                 mip_table=['Omon'],
                 ensemble_member=["r1i1p1f1"],
                 model=['IPSL-CM6A-LR'],#,'GFDL-ESM4'],
                 grid_label=['gn'],
                 variable=["zos"])

cat_Omon_zos = col.search(**query_Omon_zos)
cat_Omon_zos_lat = latest_version(cat_Omon_zos)


cat_Omon_zos_latest = intake.open_esm_datastore(cat_Omon_zos_lat,esmcol_data=esmcol_data)

dset_dict_zos = cat_Omon_zos_latest.to_dataset_dict(cdf_kwargs={'decode_times': True,'chunks': {'time': 1}}, storage_options={'anon':True},
                                                   )


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


/srv/conda/envs/notebook/lib/python3.8/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.8/site-packages/numpy/core/_asarr

In [55]:
ds_zos_ipsl_abrupt_r = dset_dict_zos["CMIP6.IPSL.IPSL-CM6A-LR.abrupt-4xCO2.Omon"]

ds_zos_ipsl_abrupt_r.time # sel(time=slice("2150","2151"))

<xarray.DataArray 'time' (time: 10800)>
array([Timestamp('1850-01-16 12:00:00'), Timestamp('1850-02-15 00:00:00'),
       Timestamp('1850-03-16 12:00:00'), ...,
       cftime.DatetimeGregorian(2749, 10, 16, 12, 0, 0, 0),
       cftime.DatetimeGregorian(2749, 11, 16, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2749, 12, 16, 12, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 1850-01-16T12:00:00 ... 2749-12-16 12:00:00
Attributes:
    axis:           T
    standard_name:  time
    long_name:      Time axis
    bounds:         time_bounds

In [58]:
ds_zos_ipsl_abrupt_r['time']#.sel(time=slice("1850","1850"))

<xarray.DataArray 'time' (time: 10800)>
array([Timestamp('1850-01-16 12:00:00'), Timestamp('1850-02-15 00:00:00'),
       Timestamp('1850-03-16 12:00:00'), ...,
       cftime.DatetimeGregorian(2749, 10, 16, 12, 0, 0, 0),
       cftime.DatetimeGregorian(2749, 11, 16, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2749, 12, 16, 12, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 1850-01-16T12:00:00 ... 2749-12-16 12:00:00
Attributes:
    axis:           T
    standard_name:  time
    long_name:      Time axis
    bounds:         time_bounds